# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 3:00PM,243302,10,8351565,"Snap Medical Industries, LLC",Released
1,Aug 2 2022 2:58PM,243301,16,8352209,Sartorius Bioprocess Solutions,Released
2,Aug 2 2022 2:56PM,243300,16,AIC-379148,American International Chemical,Released
3,Aug 2 2022 2:41PM,243299,10,MSP212738,"Methapharm, Inc.",Released
4,Aug 2 2022 2:41PM,243299,10,MSP212740,"Methapharm, Inc.",Released
5,Aug 2 2022 2:41PM,243299,10,MSP212741,"Methapharm, Inc.",Released
6,Aug 2 2022 2:41PM,243299,10,MSP212742,"Methapharm, Inc.",Released
7,Aug 2 2022 2:41PM,243299,10,MSP212751,"Methapharm, Inc.",Released
8,Aug 2 2022 2:41PM,243299,10,MSP212743,"Methapharm, Inc.",Released
9,Aug 2 2022 2:41PM,243299,10,MSP212745,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,243298,Released,14
45,243299,Released,9
46,243300,Released,1
47,243301,Released,1
48,243302,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243298,NaN,NaN,14.0
243299,NaN,NaN,9.0
243300,NaN,NaN,1.0
243301,NaN,NaN,1.0
243302,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243173,0.0,13.0,3.0
243174,14.0,5.0,18.0
243210,0.0,11.0,2.0
243217,0.0,0.0,1.0
243218,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243173,0,13,3
243174,14,5,18
243210,0,11,2
243217,0,0,1
243218,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243173,0,13,3
1,243174,14,5,18
2,243210,0,11,2
3,243217,0,0,1
4,243218,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243173,,13,3
1,243174,14,5,18
2,243210,,11,2
3,243217,,,1
4,243218,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC"
1,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions
2,Aug 2 2022 2:56PM,243300,16,American International Chemical
3,Aug 2 2022 2:41PM,243299,10,"Methapharm, Inc."
12,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation
26,Aug 2 2022 2:21PM,243297,10,Emerginnova
28,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey"
29,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation
50,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc."
51,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",,,1
1,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,,,1
2,Aug 2 2022 2:56PM,243300,16,American International Chemical,,,1
3,Aug 2 2022 2:41PM,243299,10,"Methapharm, Inc.",,,9
4,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,,,14
5,Aug 2 2022 2:21PM,243297,10,Emerginnova,,,2
6,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",,,1
7,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,,,21
8,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",,,2
9,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",1,,
1,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,1,,
2,Aug 2 2022 2:56PM,243300,16,American International Chemical,1,,
3,Aug 2 2022 2:41PM,243299,10,"Methapharm, Inc.",9,,
4,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,14,,
5,Aug 2 2022 2:21PM,243297,10,Emerginnova,2,,
6,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",1,,
7,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,21,,
8,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",2,,
9,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",1,,
1,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,1,,
2,Aug 2 2022 2:56PM,243300,16,American International Chemical,1,,
3,Aug 2 2022 2:41PM,243299,10,"Methapharm, Inc.",9,,
4,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Aug 2 2022 2:56PM,243300,16,American International Chemical,1.0,NaN,NaN
3,Aug 2 2022 2:41PM,243299,10,"Methapharm, Inc.",9.0,NaN,NaN
4,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Aug 2 2022 2:56PM,243300,16,American International Chemical,1.0,0.0,0.0
3,Aug 2 2022 2:41PM,243299,10,"Methapharm, Inc.",9.0,0.0,0.0
4,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5595519,129.0,34.0,0.0
15,486347,21.0,18.0,14.0
16,486601,2.0,0.0,0.0
19,1702705,5.0,4.0,0.0
20,1216169,9.0,44.0,17.0
21,486477,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5595519,129.0,34.0,0.0
1,15,486347,21.0,18.0,14.0
2,16,486601,2.0,0.0,0.0
3,19,1702705,5.0,4.0,0.0
4,20,1216169,9.0,44.0,17.0
5,21,486477,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,129.0,34.0,0.0
1,15,21.0,18.0,14.0
2,16,2.0,0.0,0.0
3,19,5.0,4.0,0.0
4,20,9.0,44.0,17.0
5,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,129.0
1,15,Released,21.0
2,16,Released,2.0
3,19,Released,5.0
4,20,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,14.0,0.0,0.0,17.0,0.0
Executing,34.0,18.0,0.0,4.0,44.0,2.0
Released,129.0,21.0,2.0,5.0,9.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,14.0,0.0,0.0,17.0,0.0
1,Executing,34.0,18.0,0.0,4.0,44.0,2.0
2,Released,129.0,21.0,2.0,5.0,9.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,14.0,0.0,0.0,17.0,0.0
1,Executing,34.0,18.0,0.0,4.0,44.0,2.0
2,Released,129.0,21.0,2.0,5.0,9.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()